<a href="https://colab.research.google.com/github/Svein-Tore/colab/blob/main/Lambda_tuning_interaktiv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interaktiv λ‑tuning for FOPDT‑prosess (PI‑regulator)

Denne notatboken lar deg utforske **Lambda-/Skogestad‑metoden** ved å justere prosessparametrene **K**, **T**, **L** og ønsket lukket sløyfe‑tidskonstant **λ** med sliders.

**Formler (fra undervisningsopplegget):**
- $K_p = \dfrac{T}{K(\lambda + L)}$
- $T_i = \min\big(T,\;4(\lambda + L)\big)$

Kjør cellene fra toppen og bruk sliderne til å se hvordan **PV** og **pådrag u** endrer seg.

In [5]:
import numpy as np
import matplotlib.pyplot as plt

# Forsøk å bruke ipywidgets for best interaktivitet i Jupyter
try:
    import ipywidgets as widgets
    from ipywidgets import FloatSlider, HBox, VBox
    HAS_W = True
except Exception:
    HAS_W = False

plt.rcParams['figure.dpi'] = 120
plt.rcParams['font.size'] = 11
# --- Simuleringskjerne: FOPDT + dødtid + PI (lambda-tunet) ---

def simulate_fopdt_pi(K=2.904, T=125.4, L=5.2, lam=62.7, dt=0.1, T_end=300.0,
                      u_clip=2.0):
    """Simulerer lukket sløyfe for en FOPDT-prosess med PI-regulator.
    Returnerer tid, PV (y), pådrag (u), samt (Kp, Ti).
    """
    # PI-parametre fra Lambda-metoden
    Kp = T / (K * (lam + L))
    Ti = min(T, 4.0 * (lam + L))

    time = np.arange(0.0, T_end + dt, dt)
    sp = np.ones_like(time)  # enhetssteg

    y = np.zeros_like(time)
    u = np.zeros_like(time)
    e_int = 0.0

    delay_steps = int(np.round(L/dt))
    u_buf = np.zeros(delay_steps + 1)

    for k in range(1, len(time)):
        e = sp[k] - y[k-1]
        e_int += e * dt
        u[k] = Kp * (e + (e_int / Ti))
        if u_clip is not None:
            u[k] = np.clip(u[k], -abs(u_clip), abs(u_clip))

        # dødtid på u
        if delay_steps > 0:
            u_buf = np.roll(u_buf, 1)
            u_buf[0] = u[k]
            u_delay = u_buf[-1]
        else:
            u_delay = u[k]

        # FOPDT: dy/dt = (-y + K*u_delay)/T
        dy = (-y[k-1] + K * u_delay) / T
        y[k] = y[k-1] + dy * dt

    return time, sp, y, u, Kp, Ti
# Standardverdier (fra nivå-eksemplet i undervisningsopplegget)
K0, T0, L0 = 2.904, 125.4, 5.2
lam0 = T0/2  # rolig standard

if HAS_W:
    K_sl = FloatSlider(value=K0, min=0.1, max=10.0, step=0.01, description='K')
    T_sl = FloatSlider(value=T0, min=5.0, max=500.0, step=0.1, description='T [s]')
    L_sl = FloatSlider(value=L0, min=0.0, max=60.0, step=0.1, description='L [s]')
    lam_sl = FloatSlider(value=lam0, min=0.1, max=400.0, step=0.1, description='λ [s]')

    dt_sl = FloatSlider(value=0.1, min=0.02, max=0.5, step=0.01, description='Δt [s]')
    Tend_sl = FloatSlider(value=300.0, min=30.0, max=1200.0, step=10.0, description='T_end [s]')
    uclip_sl = FloatSlider(value=2.0, min=0.5, max=5.0, step=0.1, description='|u|max')

    out_fig = widgets.Output()

    def update_plot(change=None):
        with out_fig:
            out_fig.clear_output(wait=True)
            time, sp, y, u, Kp, Ti = simulate_fopdt_pi(K_sl.value, T_sl.value, L_sl.value,
                                                       lam_sl.value, dt_sl.value, Tend_sl.value,
                                                       u_clip=uclip_sl.value)
            fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(7.2, 6), sharex=True,
                                           gridspec_kw={'height_ratios':[2.0,1.0]})
            ax1.plot(time, sp, 'k--', lw=1.2, label='SP')
            ax1.plot(time, y, lw=2.0, label='PV')
            ax1.set_ylabel('PV [–]'); ax1.grid(True, alpha=0.3); ax1.legend(loc='lower right')
            ax1.set_title('FOPDT + PI (λ‑tuning)  |  ' +
                          f"Kp={Kp:.3f}, Ti={Ti:.1f}s  |  λ={lam_sl.value:.1f}s, T={T_sl.value:.1f}s, L={L_sl.value:.1f}s")

            ax2.plot(time, u, lw=1.6)
            ax2.set_ylabel('u [–]'); ax2.set_xlabel('Tid [s]'); ax2.grid(True, alpha=0.3)
            plt.show()

    for w in (K_sl, T_sl, L_sl, lam_sl, dt_sl, Tend_sl, uclip_sl):
        w.observe(update_plot, names='value')

    update_plot()
    display(VBox([
        HBox([K_sl, T_sl]),
        HBox([L_sl, lam_sl]),
        HBox([dt_sl, Tend_sl, uclip_sl]),
        out_fig
    ]))
else:
    # Fallback uten ipywidgets: juster verdiene manuelt under og kjør cellen hver gang
    K, T, L, lam = K0, T0, L0, lam0
    dt, T_end, u_clip = 0.1, 300.0, 2.0
    time, sp, y, u, Kp, Ti = simulate_fopdt_pi(K, T, L, lam, dt, T_end, u_clip)
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(7.2, 6), sharex=True,
                                   gridspec_kw={'height_ratios':[2.0,1.0]})
    ax1.plot(time, sp, 'k--', lw=1.2, label='SP')
    ax1.plot(time, y, lw=2.0, label='PV')
    ax1.set_ylabel('PV [–]'); ax1.grid(True, alpha=0.3); ax1.legend(loc='lower right')
    ax1.set_title(f'FOPDT + PI (λ‑tuning)  |  Kp={Kp:.3f}, Ti={Ti:.1f}s  |  λ={lam:.1f}s, T={T:.1f}s, L={L:.1f}s')
    ax2.plot(time, u, lw=1.6)
    ax2.set_ylabel('u [–]'); ax2.set_xlabel('Tid [s]'); ax2.grid(True, alpha=0.3)
    plt.show()
